In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 450.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=14dadeeec7600c3dd70739d6cb3c906d1e54b0c2dc1e7f966d3380fc4bc0dcd8
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --no-deps

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.3 MB/s eta 0:00:00


In [ ]:
import numpy
print("Numpy version:", numpy.__version__)

import torch
print("CUDA available:", torch.cuda.is_available())

Numpy version: 1.26.4
CUDA available: True


In [ ]:
!mkdir -p /content/VisDrone_Subset/images/train
!mkdir -p /content/VisDrone_Subset/images/val
!mkdir -p /content/VisDrone_Subset/labels/train
!mkdir -p /content/VisDrone_Subset/labels/val

!rsync -ah --progress '/content/drive/MyDrive/Subset/images/train/' /content/VisDrone_Subset/images/train/
!rsync -ah --progress '/content/drive/MyDrive/Subset/labels/train/' /content/VisDrone_Subset/labels/train/
!rsync -ah --progress '/content/drive/MyDrive/Subset/images/val/' /content/VisDrone_Subset/images/val/
!rsync -ah --progress '/content/drive/MyDrive/Subset/labels/val/' /content/VisDrone_Subset/labels/val/

流式输出内容被截断，只能显示最后 5000 行内容。
0000175_00001_d_0000001.txt
            152 100%    0.27kB/s    0:00:00 (xfr#103, to-chk=1897/2001)
0000175_00432_d_0000001.txt
             38 100%    0.07kB/s    0:00:00 (xfr#104, to-chk=1896/2001)
0000176_00001_d_0000001.txt
            304 100%    0.53kB/s    0:00:00 (xfr#105, to-chk=1895/2001)
0000177_00000_d_0000001.txt
            266 100%    0.46kB/s    0:00:00 (xfr#106, to-chk=1894/2001)
0000177_00374_d_0000003.txt
             76 100%    0.13kB/s    0:00:00 (xfr#107, to-chk=1893/2001)
0000177_00570_d_0000004.txt
            114 100%    0.20kB/s    0:00:00 (xfr#108, to-chk=1892/2001)
0000178_00201_d_0000007.txt
            266 100%    0.46kB/s    0:00:00 (xfr#109, to-chk=1891/2001)
0000178_00621_d_0000009.txt
             38 100%    0.07kB/s    0:00:00 (xfr#110, to-chk=1890/2001)
0000179_00301_d_0000015.txt
            114 100%    0.19kB/s    0:00:00 (xfr#111, to-chk=1889/2001)
0000179_00445_d_0000016.txt
            342 100%    0.58kB/s    0:00:00 (

In [ ]:
import os, shutil, random

img_dir = '/content/VisDrone_Subset/images/train'
label_dir = '/content/VisDrone_Subset/labels/train'

for i in range(5):
    os.makedirs(f'/content/client{i}/images', exist_ok=True)
    os.makedirs(f'/content/client{i}/labels', exist_ok=True)

all_imgs = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])
random.shuffle(all_imgs)
split_size = len(all_imgs) // 5

for i in range(5):
    subset = all_imgs[i*split_size : (i+1)*split_size]
    for img in subset:
        label = img.rsplit('.', 1)[0] + '.txt'
        shutil.copy(os.path.join(img_dir, img), f'/content/client{i}/images/{img}')
        shutil.copy(os.path.join(label_dir, label), f'/content/client{i}/labels/{label}')

In [ ]:
names = ['car', 'van', 'truck']
nc = len(names)

for i in range(5):
    yaml_text = f"""
path: /content/client{i}
train: images
val: /content/VisDrone_Subset/images/val
nc: {nc}
names: {names}
"""
    with open(f"/content/client{i}.yaml", "w") as f:
        f.write(yaml_text.strip())

In [ ]:
#FGSM test
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import os

model_path = "/content/drive/MyDrive/fed_model2.pt"

save_dir = "/content/drive/MyDrive/AdversarialAttack"
os.makedirs(save_dir, exist_ok=True)
img_path = "/content/VisDrone_Subset/images/val/0000001_05999_d_0000011.jpg"

model = YOLO(model_path).to('cuda')
print("Model Loaded Successfully")

orig = cv2.imread(img_path)
img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
img_tensor = torch.tensor(img).permute(2,0,1).unsqueeze(0).float().cuda()/255

epsilon = 8/255

resized = cv2.resize(img, (640, 640))
input_tensor = torch.tensor(resized).permute(2, 0, 1).unsqueeze(0).float().cuda() / 255
input_tensor.requires_grad = True

outputs = model.model(input_tensor)
if isinstance(outputs, (list, tuple)):
    total = 0
    for out in outputs:
        if isinstance(out, torch.Tensor):
            total += out.sum()
    loss = total
else:
    loss = outputs.sum()

loss.backward()
perturbed_img = input_tensor + epsilon * input_tensor.grad.sign()
perturbed_img = torch.clamp(perturbed_img, 0, 1).detach()
adv_np = (perturbed_img.squeeze().permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
adv_path = os.path.join(save_dir, "adv.jpg")
cv2.imwrite(adv_path, cv2.cvtColor(adv_np, cv2.COLOR_RGB2BGR))

print("The adversarial sample saved：", adv_path)

print("Original image")
model.predict(img_path, conf=0.25, save=True, name="predict_clean")
print("Adversarial image")
model.predict(adv_path, conf=0.25, save=True, name="predict_adv")

模型加载完成！
对抗样本已保存： /content/drive/MyDrive/AdversarialAttack/adv.jpg
推理原始图片...

image 1/1 /content/VisDrone_Subset/images/val/0000001_05999_d_0000011.jpg: 384x640 44 cars, 1 van, 63.8ms
Speed: 2.6ms preprocess, 63.8ms inference, 313.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict_clean
推理对抗图片...

image 1/1 /content/drive/MyDrive/AdversarialAttack/adv.jpg: 640x640 22 cars, 26.2ms
Speed: 4.2ms preprocess, 26.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict_adv


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'car', 1: 'van', 2: 'truck'}
 obb: None
 orig_img: array([[[159, 163, 151],
         [146, 148, 136],
         [158, 153, 144],
         ...,
         [102, 111, 115],
         [ 67,  74,  71],
         [ 36,  44,  34]],
 
        [[158, 160, 148],
         [153, 152, 142],
         [164, 158, 151],
         ...,
         [ 85,  94,  97],
         [ 58,  66,  59],
         [ 38,  46,  35]],
 
        [[148, 144, 133],
         [120, 115, 106],
         [116, 109, 106],
         ...,
         [ 89,  97,  96],
         [ 41,  51,  39],
         [ 32,  42,  25]],
 
        ...,
 
        [[ 87,  95, 102],
         [ 90,  97, 100],
         [ 77,  83,  82],
         ...,
         [ 73, 101,  81],
         [ 90, 118,  99],
         [ 78, 106,  87]],
 
        [[ 74,  85,  89],
         [ 77,  86,  89],
         [ 68,  76,  75],
         ...,

In [ ]:
#FGSM_official
import os
import cv2
import csv
import torch
import random
import numpy as np
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/fed_model2.pt"
val_img_dir = "/content/VisDrone_Subset/images/val/"
save_dir = "/content/drive/MyDrive/AdversarialAttack/FGSM"
os.makedirs(save_dir, exist_ok=True)

epsilon = 8/255
num_samples = 5

all_images = [os.path.join(val_img_dir, f) for f in os.listdir(val_img_dir) if f.endswith('.jpg')]
random.shuffle(all_images)
image_files = all_images[:num_samples]

model = YOLO(model_path).to("cuda")

def generate_fgsm(model, img_path, epsilon=8/255):
    img0 = cv2.imread(img_path)
    img = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (640, 640))
    x = torch.tensor(img_resized).permute(2,0,1).unsqueeze(0).float().cuda()/255
    x.requires_grad = True
    model.model.train()

    with torch.enable_grad():
        outputs = model.model(x)

    model.model.eval()
    loss = 0
    for o in outputs:
        if isinstance(o, torch.Tensor):
            loss += o.sum()
    loss.backward()

    adv = x + epsilon * x.grad.sign()
    adv = torch.clamp(adv, 0, 1).detach()

    adv_np = (adv.squeeze().permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
    return adv_np

results_csv = os.path.join(save_dir, "fgsm_random5_results.csv")
with open(results_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Clean_boxes", "FGSM_boxes"])

    for img_path in image_files:
        print(f"\n处理图像: {img_path}")
        adv_np = generate_fgsm(model, img_path, epsilon)
        adv_path = os.path.join(save_dir, "fgsm_" + os.path.basename(img_path))
        cv2.imwrite(adv_path, cv2.cvtColor(adv_np, cv2.COLOR_RGB2BGR))
        res_clean = model.predict(img_path, conf=0.25, save=True, name="predict_clean_random5")
        n_clean = len(res_clean[0].boxes)
        res_adv = model.predict(adv_path, conf=0.25, save=True, name="predict_adv_random5")
        n_adv = len(res_adv[0].boxes)

        writer.writerow([os.path.basename(img_path), n_clean, n_adv])
        print(f"Clean boxes: {n_clean}, FGSM boxes: {n_adv}")
    f.close()

print(f"\nThe result save to: {results_csv}")


处理图像: /content/VisDrone_Subset/images/val/0000242_03668_d_0000012.jpg

image 1/1 /content/VisDrone_Subset/images/val/0000242_03668_d_0000012.jpg: 384x640 12 cars, 1 van, 11.6ms
Speed: 2.9ms preprocess, 11.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict_clean_random5

image 1/1 /content/drive/MyDrive/AdversarialAttack/FGSM/fgsm_0000242_03668_d_0000012.jpg: 640x640 5 cars, 1 van, 1 truck, 17.0ms
Speed: 1.9ms preprocess, 17.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict_adv_random5
Clean boxes: 13, FGSM boxes: 7

处理图像: /content/VisDrone_Subset/images/val/0000249_00001_d_0000001.jpg

image 1/1 /content/VisDrone_Subset/images/val/0000249_00001_d_0000001.jpg: 384x640 13 cars, 2 vans, 11.7ms
Speed: 2.0ms preprocess, 11.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict_clean_random52

image 1/1 /content/drive/MyDrive/Adver

In [ ]:
#FGSM_metris
adv_yaml_path = "/content/adv_val.yaml"

yaml_text = """
path: /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_adv
train: images
val: images
nc: 3
names: ['car', 'van', 'truck']
"""

with open(adv_yaml_path, "w") as f:
    f.write(yaml_text.strip())

print("Grnerate new yaml file:", adv_yaml_path)

重新生成的 yaml 文件: /content/adv_val.yaml


In [ ]:
model = YOLO(model_path).to("cuda")
print("Evaluate clean val")
metrics_clean = model.val(
    data="client0.yaml",
    imgsz=640,
    conf=0.25
)
print(metrics_clean)

print("Evaluate adversarial images of FGSM")
metrics_adv = model.val(
    data="adv_val.yaml",
    imgsz=640,
    conf=0.25
)
print(metrics_adv)

def extract_metrics(metrics):
    return {
        "mAP50": metrics.results_dict['metrics/mAP50(B)'],
        "mAP50-95": metrics.results_dict['metrics/mAP50-95(B)'],
        "Precision": metrics.results_dict['metrics/precision(B)'],
        "Recall": metrics.results_dict['metrics/recall(B)'],
    }

clean_values = extract_metrics(metrics_clean)
adv_values = extract_metrics(metrics_adv)

print("\nComparison attack before and after attack")
print(f"{'Index':<12}{'Clean':>12}{'FGSM':>12}")
for key in clean_values:
    print(f"{key:<12}{clean_values[key]:>12.4f}{adv_values[key]:>12.4f}")


评估干净验证集
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1982.5±661.0 MB/s, size: 152.5 KB)


val: Scanning /content/VisDrone_Subset/labels/val.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  3.91it/s]


                   all        300       5128      0.657      0.592      0.647       0.51
                   car        294       4280       0.79      0.847      0.866        0.7
                   van        216        616      0.561      0.563      0.583       0.46
                 truck        122        232      0.619      0.366      0.492       0.37
Speed: 0.7ms preprocess, 5.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val5
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b5297fff50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0

val: Scanning /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_adv/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.14it/s]


                   all          5         56      0.333     0.0256      0.179       0.15
                   car          5         39          1     0.0769      0.538       0.45
                   van          5         11          0          0          0          0
                 truck          4          6          0          0          0          0
Speed: 0.2ms preprocess, 17.2ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to runs/detect/val6
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b4e4c83c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    

In [ ]:
!rm -rf /content/runs

In [ ]:
!rm -rf /content/drive/MyDrive/AdversarialAttack/FGSM/runs

In [ ]:
!mkdir -p /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_clean
!mkdir -p /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_adv

!cp runs/detect/predict_clean_random5*/*.jpg /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_clean/
!cp runs/detect/predict_adv_random5*/*.jpg /content/drive/MyDrive/AdversarialAttack/FGSM/runs/all_adv/

In [ ]:
#PGD
import os
import cv2
import csv
import torch
import random
import numpy as np
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/fed_model2.pt"
val_img_dir = "/content/VisDrone_Subset/images/val/"
save_dir = "/content/drive/MyDrive/AdversarialAttack/PGD"
os.makedirs(save_dir, exist_ok=True)

epsilon = 8/255
alpha = 2/255
pgd_steps = 10
num_samples = 5

all_images = [os.path.join(val_img_dir, f) for f in os.listdir(val_img_dir) if f.endswith('.jpg')]
random.shuffle(all_images)
image_files = all_images[:num_samples]
model = YOLO(model_path).to("cuda")

def generate_pgd(model, img_path, epsilon=8/255, alpha=2/255, steps=10):
    img0 = cv2.imread(img_path)
    img = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (640, 640))

    x_orig = torch.tensor(img_resized).permute(2,0,1).unsqueeze(0).float().cuda()/255
    adv = x_orig.clone().detach()

    for _ in range(steps):
        adv.requires_grad = True
        model.model.train()

        with torch.enable_grad():
            outputs = model.model(adv)
        model.model.eval()
        loss = 0
        for o in outputs:
            if isinstance(o, torch.Tensor):
                loss += o.sum()

        loss.backward()

        adv = adv + alpha * adv.grad.sign()

        delta = torch.clamp(adv - x_orig, min=-epsilon, max=epsilon)
        adv = torch.clamp(x_orig + delta, 0, 1).detach()

    adv_np = (adv.squeeze().permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
    return adv_np

results_csv = os.path.join(save_dir, "pgd_random5_results.csv")
with open(results_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Clean_boxes", "PGD_boxes"])

    for img_path in image_files:
        print(f"\nProcess images: {img_path}")
        adv_np = generate_pgd(model, img_path, epsilon, alpha, pgd_steps)
        adv_path = os.path.join(save_dir, "pgd_" + os.path.basename(img_path))

        cv2.imwrite(adv_path, cv2.cvtColor(adv_np, cv2.COLOR_RGB2BGR))

        res_clean = model.predict(img_path, conf=0.25, save=True, name="predict_clean_random5_pgd")
        n_clean = len(res_clean[0].boxes)
        res_adv = model.predict(adv_path, conf=0.25, save=True, name="predict_adv_random5_pgd")
        n_adv = len(res_adv[0].boxes)

        writer.writerow([os.path.basename(img_path), n_clean, n_adv])
        print(f"Clean boxes: {n_clean}, PGD boxes: {n_adv}")
    f.close()

print(f"\nThe result saved to: {results_csv}")



处理图像: /content/VisDrone_Subset/images/val/0000335_02549_d_0000056.jpg

image 1/1 /content/VisDrone_Subset/images/val/0000335_02549_d_0000056.jpg: 384x640 14 cars, 1 van, 10.8ms
Speed: 2.9ms preprocess, 10.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict_clean_random5_pgd

image 1/1 /content/drive/MyDrive/AdversarialAttack/PGD/pgd_0000335_02549_d_0000056.jpg: 640x640 11 cars, 2 vans, 11.3ms
Speed: 2.7ms preprocess, 11.3ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict_adv_random5_pgd
Clean boxes: 15, PGD boxes: 13

处理图像: /content/VisDrone_Subset/images/val/0000103_01734_d_0000028.jpg

image 1/1 /content/VisDrone_Subset/images/val/0000103_01734_d_0000028.jpg: 384x640 1 car, 8.4ms
Speed: 3.1ms preprocess, 8.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict_clean_random5_pgd2

image 1/1 /content/drive/MyDrive/AdversarialAtt

In [ ]:
model = YOLO(model_path).to("cuda")

print("Evaluate clean val")
metrics_clean = model.val(
    data="client0.yaml",
    imgsz=640,
    conf=0.25
)
print(metrics_clean)

print("Evaluate adversarial images of PGD")
metrics_adv = model.val(
    data="adv_val.yaml",
    imgsz=640,
    conf=0.25
)
print(metrics_adv)

def extract_metrics(metrics):
    return {
        "mAP50": metrics.results_dict['metrics/mAP50(B)'],
        "mAP50-95": metrics.results_dict['metrics/mAP50-95(B)'],
        "Precision": metrics.results_dict['metrics/precision(B)'],
        "Recall": metrics.results_dict['metrics/recall(B)'],
    }

clean_values = extract_metrics(metrics_clean)
adv_values = extract_metrics(metrics_adv)
print("\nComparison attack before and after (PGD)")
print(f"{'Index':<12}{'Clean':>12}{'PGD':>12}")
for key in clean_values:
    print(f"{key:<12}{clean_values[key]:>12.4f}{adv_values[key]:>12.4f}")


评估干净验证集
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2492.7±458.0 MB/s, size: 134.1 KB)


val: Scanning /content/VisDrone_Subset/labels/val.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  3.97it/s]


                   all        300       5128      0.657      0.592      0.647       0.51
                   car        294       4280       0.79      0.847      0.866        0.7
                   van        216        616      0.561      0.563      0.583       0.46
                 truck        122        232      0.619      0.366      0.492       0.37
Speed: 0.9ms preprocess, 5.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b4c42f5350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0

val: Scanning /content/drive/MyDrive/AdversarialAttack/PGD/runs/all_adv/labels... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 162.89it/s]

val: New cache created: /content/drive/MyDrive/AdversarialAttack/PGD/runs/all_adv/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.30it/s]


                   all          5         56      0.333     0.0128      0.173       0.13
                   car          5         52          1     0.0385      0.519      0.389
                   van          1          1          0          0          0          0
                 truck          2          3          0          0          0          0
Speed: 0.2ms preprocess, 15.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val4
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b4e410c0d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    